
# Modelling & solving the multiple couriers planning problem

Matthieu Feraud and Marius Lesaulnier | Erasmus 2022 - 2023


---

## Installations and Imports

In [370]:
'''
!pip3 install z3-solver
!pip3 install utils
!pip3 install gurobipy
'''

'\n!pip3 install z3-solver\n!pip3 install utils\n!pip3 install gurobipy\n'

In [17]:
from os import scandir, getcwd
import csv
import numpy as np
from math import log2
from z3.z3 import *
import time
from gurobipy import Model, GRB, quicksum

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [7]:
filespath = getcwd() + '\\Instances\\instancesDat\\'
data_list = []

with scandir(filespath) as file_list:
    for filename in file_list:
        with open(filespath + filename.name) as f:
            lines = f.readlines()

            num_couriers = int(lines[0].strip())
            num_items = int(lines[1].strip())
            courier_loads = [int(i) for i in lines[2].strip().split()]
            item_sizes = [int(i) for i in lines[3].strip().split()]
            distance_matrix = [[int(j) for j in i.strip().split()] for i in lines[4:]]

            data_list.append({
                "m": num_couriers,
                "n": num_items,
                "l": courier_loads,
                "s": item_sizes,
                "D": distance_matrix
            })

In [373]:
data_list = sorted(data_list, key=lambda x: x['m'] + x['n'])
for i in range(20):
  data = data_list[i]
  m = data['m']  # num_couriers
  n = data['n']  # num_items
  print("id: "+str(i)+" m: "+str(m), "n: "+str(n), data['D'][0])

id: 0 m: 2 n: 3 [0, 21, 86, 99]
id: 1 m: 2 n: 6 [0, 3, 4, 5, 6, 6, 2]
id: 2 m: 3 n: 7 [0, 3, 3, 6, 5, 6, 6, 2]
id: 3 m: 6 n: 8 [0, 80, 131, 22, 41, 127, 87, 48, 113]
id: 4 m: 6 n: 9 [0, 199, 119, 28, 179, 77, 145, 61, 123, 87]
id: 5 m: 8 n: 10 [0, 56, 86, 87, 81, 128, 107, 163, 166, 98, 93]
id: 6 m: 8 n: 10 [0, 56, 86, 77, 81, 128, 107, 154, 70, 93, 53]
id: 7 m: 10 n: 13 [0, 49, 80, 59, 112, 79, 112, 187, 28, 47, 36, 69, 138, 54]
id: 8 m: 10 n: 13 [0, 21, 86, 14, 84, 72, 24, 54, 83, 70, 8, 91, 42, 57]
id: 9 m: 6 n: 17 [0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66]
id: 10 m: 3 n: 47 [0, 60, 141, 22, 41, 137, 77, 48, 92, 105, 113, 103, 82, 15, 79, 24, 98, 69, 82, 30, 105, 89, 57, 94, 75, 50, 127, 16, 36, 77, 57, 70, 51, 101, 88, 38, 83, 108, 81, 124, 54, 131, 99, 70, 112, 162, 94, 64]
id: 11 m: 20 n: 47 [0, 59, 135, 21, 41, 132, 75, 48, 88, 102, 109, 101, 81, 15, 78, 24, 96, 67, 80, 30, 103, 87, 56, 93, 73, 49, 125, 15, 36, 76, 56, 68, 50, 98, 85, 37, 81, 106, 79

In [8]:
data = data_list[6]
m = data['m']  # num_couriers
n = data['n']  # num_items
l = data['l']  # courier_loads
s = data['s']  # item_sizes
D = data['D']  # distance_matrix

## SAT MODEL

### Defining the decisions variables and constraints

In [ ]:
def define_decision_variables(num_couriers, num_items):
    #These variables control the couriers’ movements and ensure that they follow the optimal path
    x = [[Bool(f"x_{i}_{j}") for j in range(num_items+1)] for i in range(num_items+1)]
    #These variables signal ’true’ when courier k is allocated to node i
    v = [[Bool(f"v_{i}_{k}") for k in range(num_couriers)] for i in range(num_items)]

    num_bits = int(log2(num_items))+1
    #These variables purpose is to encode the sequence of node visits in binary notation
    u = [[Bool(f"u_{i}_{k}") for k in range(num_bits)] for i in range(num_items)]
    return x, v, u

In [ ]:
def exactly_one(boolean_vars):
    return And(AtMost(*boolean_vars, 1), AtLeast(*boolean_vars, 1))

# enforce the ordering of visiting locations in the solution.
# It ensures that if location i is visited before location j, then the binary representation
# of i should be less than the binary representation of j.
def binary_increment(a, b):
    constraints = []
    carry = {}
    num_digits = len(a)

    constraints.append(b[0] == Not(a[0]))
    constraints.append(b[1] == Or(And(a[1],Not(a[0])), And(Not(a[1]),a[0])))
    carry[1] = a[0]

    for i in range(2,num_digits):
        carry[i] = And(a[i-1],carry[i-1])
        constraints.append(b[i] == Or(And(a[i],Not(carry[i])),And(Not(a[i]),carry[i])))

    return And(constraints)


In [10]:
def add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([x[i][i] == False for i in range(num_items+1)])

    # Each location must be visited exactly once
    for i in range(num_items):
        solver.add(exactly_one([x[i][j] for j in range(num_items+1)]))
        solver.add(exactly_one([x[j][i] for j in range(num_items+1)]))

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add((Sum([x[num_items][j] for j in range(num_items)])== num_couriers))
    solver.add((Sum([x[i][num_items] for i in range(num_items)])== num_couriers))

    solver.add([[x[j][i] == False for i in range(num_items)] for j in range(num_items) if i < j and distance_matrix[i,j] == distance_matrix[j,i] ])

    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(num_couriers) for i in range(num_items) for j in range(num_items)])

    # Each item must be carried by exactly one courier
    for i in range(num_items):
        solver.add(exactly_one([v[i][k] for k in range(num_couriers)]))

    # The total size of items carried by each courier must not exceed the courier's max load
    # PbLe to restrict the weighted sum of each items of each couriers to be less or equal to their max_loads
    for k in range(num_couriers):
        solver.add(PbLe([(v[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(PbLe([(x[i][j], distance_matrix[i, j]) for j in range(num_items+1) for i in range(num_items+1) if i!=j], max_distance-1))

    # # The order of visiting locations must be consistent with the binary representations

    # for j in range(num_items):
    #     for i in range(num_items):
    #         if i!=j:
    #             solver.add(Implies(x[i][j], binary_increment(u[i], u[j])))

    # Symmetry-breaking constraint: enforce a specific ordering of courier assignments
    # for i in range(num_items):
    #     solver.add([Implies(v[i][k2], v[i][k1]) for k1 in range(num_couriers) for k2 in range(num_couriers) if k1 > k2 and max_loads[k1]== max_loads[k2]])

    # # Symmetry-breaking constraint: If courier i goes from the origin to location j, then i must be less than j, order of node assignment
    # solver.add([Implies(And(x[num_items][i],x[i][j]),i<j) for i in range(num_items) for j in range(num_items) if i!=j])

    # Each courier must carry at least one item
    solver.add([AtLeast(*[v[i][k] for i in range(num_items)], 1) for k in range(num_couriers)])



### Formulating the Optimization Problem

In [11]:
def define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance=None):
    distance_matrix = np.array(distance_matrix)

    if max_distance is None:
        max_distance = int(np.sum(distance_matrix))

    #solver = Optimize()
    solver = SolverFor("QF_BV")

    x, v, u = define_decision_variables(num_couriers, num_items)

    add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance)

    #total_distance = Sum([distance_matrix[i, j] * x[i][j] for j in range(num_items + 1) for i in range(num_items + 1) if i != j])
    #solver.minimize(total_distance)
    return solver, x, v




### Interpreting and Displaying Optimization Results

In [12]:
def handle_solution(solver, x, v, num_couriers, num_items, distance_matrix):
    solver.set("timeout", 60)  # timeout is in milliseconds
    model = solver.model()
    route_matrix = [[model.evaluate(x[i][j]) for j in range(num_items + 1)] for i in range(num_items + 1)]
    item_courier_matrix = [[model.evaluate(v[i][k]) for k in range(num_couriers)] for i in range(num_items)]

    for i in range(len(v)):
      assigned_couriers = []
      for j in range(len(v[0])):
          if model.evaluate(v[i][j]):
              assigned_couriers.append(j+1)
      if assigned_couriers:
          print("Item", i+1, "assigned to", assigned_couriers)
      else:
          print("Item", i+1, "not assigned to any courier")


    num_nodes = len(route_matrix) - 1
    routes = {}
    courier_distance = {}

    for origin_node in range(num_nodes + 1):
        if route_matrix[num_nodes][origin_node]:  # if there is a route from the last node to the current node
            courier_path = [num_nodes, origin_node]  # start a new path
            courier_id = item_courier_matrix[origin_node].index(True)  # assign a courier to this path
            print("courier id: ", courier_id)
            current_node = origin_node  # initialize the current node
            courier_distance[courier_id] = distance_matrix[num_nodes][origin_node]  # initialize the courier's total distance

            # Continue the path until we reach back to the last node
            while current_node != num_nodes:
                if courier_id != item_courier_matrix[current_node].index(True):
                    print('Some error occurred with courier', courier_id)
                for next_node in range(num_nodes + 1):
                    if current_node != next_node and route_matrix[current_node][next_node]:
                        courier_path.append(next_node)  # append the next node to the path
                        courier_distance[courier_id] += distance_matrix[current_node][next_node]  # update the courier's total distance
                        current_node = next_node  # move to the next node
                        break
            routes[courier_id] = courier_path  # store the courier's path
    total_distance = 0
    paths = []

    for courier_id, route in routes.items():
        path_display = " > ".join([str((node + 1) % (num_nodes + 1)) for node in route])
        distance_display = f"{courier_distance[courier_id]:,}"
        paths.append((courier_id + 1, path_display, distance_display))
        total_distance += courier_distance[courier_id]

    # Print the courier paths
    print("Courier Paths:")
    print("-" * 60)
    print("{:<10s} {:<30s} {:<15s}".format("Courier", "Path", "Distance"))
    print("-" * 60)
    for courier_id, path_display, distance_display in paths:
        print("{:<10d} {:<30s} {:<15s}".format(courier_id, path_display, distance_display))
    print("-" * 60)

    # Print the total distance
    print("Total distance:", f"{total_distance:,}")
    return total_distance


### Executing Item Assignment and Route Optimization for the SAT Model

In [13]:
def assign(num_couriers, num_items, max_loads, item_sizes, distance_matrix):
    if sum(item_sizes) > sum(max_loads):
        print("Total weight of all items surpasses possible weight transportable")
        return 0

    remaining_time = 300
    solver, x, v = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix)

    if solver.check() == sat:
        total_distance = handle_solution(solver, x, v, num_couriers, num_items, distance_matrix)
        print(f"Initial solution found with total distance: {total_distance}")

        while remaining_time > 0:
            print(f"Looking for solutions with total distance < {total_distance}")
            time_before = time.time()
            solver, x, v = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix, total_distance)

            if solver.check() == sat:
                total_distance = handle_solution(solver, x, v, num_couriers, num_items, distance_matrix)
                elapsed_time = time.time()-time_before
                remaining_time -= elapsed_time
                print(f"Elapsed time: {elapsed_time:.2f} seconds")
                print(f"Remaining time: {remaining_time:.2f} seconds")
            else:
                print("No more solutions found.")
                break

        if remaining_time <= 0:
            print("Time limit exceeded")
        return 0
    else:
        print("Failed to find a solution")
        return 0


In [14]:
assign(m,n,l,s,D)

NameError: name 'SolverFor' is not defined

## MIP Model

### Defining the constraints

In [366]:
# Function to add constraints to the model
def add_constraints(mdl, num_items, num_couriers, Vertices, Items, Arcs, x, l, courier_assignment, aux_vars, item_loads, max_loads):

    # Only one connection into point i
    mdl.addConstrs(quicksum(x[i,j] for j in Vertices if j != i) == 1 for i in Items)

    # Only one edge out of point i
    mdl.addConstrs(quicksum(x[i,j] for i in Vertices if i != j) == 1 for j in Items)

    # No more than "num_couriers" departures from the depot
    mdl.addConstr(quicksum(x[0,j] for j in Items ) <= num_couriers )

    # Same number of couriers leaving and arriving to the depot
    mdl.addConstr(quicksum(x[0,j] for j in Items ) == quicksum(x[j,0] for j in Items ))

    # Sub-tour elimination
    mdl.addConstr(l[0] == 1)
    mdl.addConstrs((l[i] + x[i,j]) <= (l[j] + num_items * (1 - x[i,j])) for i,j in Arcs if j!=0 )

    # Constraints related to courier
    for k in range(num_couriers):
        mdl.addConstrs((aux_vars[i][k] <= courier_assignment[i][k]) for i in Items)
        mdl.addConstrs((aux_vars[i][k] <= x[0,i]) for i in Items )
        mdl.addConstrs((aux_vars[i][k] >= courier_assignment[i][k] + x[0,i] - 1 ) for i in Items )
        mdl.addConstr(quicksum(aux_vars[i][k] for i in Items) <= 1 )

        # If x[i,j] = True, i,j share the same courier
        for i,j in Arcs:
            if i!=0 and j!=0:
                mdl.addConstr(x[i,j] + courier_assignment[i][k] - courier_assignment[j][k] <= 1 )
                mdl.addConstr(x[i,j] - courier_assignment[i][k] + courier_assignment[j][k] <= 1 )

    # Every item should be visited by exactly one courier
    mdl.addConstrs(quicksum(courier_assignment[i][k] for k in range(num_couriers)) == 1 for i in Items)

    # The total load of items served by each courier cannot exceed the courier's capacity
    mdl.addConstrs(quicksum( item_loads[i-1] * courier_assignment[i][k] for i in Items) <= max_loads[k] for k in range(num_couriers))

### Creating the model

In [367]:
# Function to create the Gurobi model
def create_model(num_items, num_couriers, item_loads, item_sizes, distance_matrix):

    Items = [i for i in range(1,num_items+1)]
    # Define the set of vertices, including the depot
    Vertices = [0] + Items
    Arcs = [(i, j) for i in Vertices for j in Vertices if i != j]

    Couriers_encoding = [i for i in range(num_couriers)]
    courier_assignment = {}
    aux_vars = {}

    mdl = Model('VRP')

    # Define variables
    x = mdl.addVars(Arcs, vtype=GRB.BINARY)
    l = mdl.addVars(Vertices, lb=0, ub=num_items, vtype=GRB.INTEGER)
    for i in range(1,num_items+1):
        courier_assignment[i] = mdl.addVars(Couriers_encoding, vtype=GRB.BINARY)
        aux_vars[i] = mdl.addVars(Couriers_encoding, vtype=GRB.INTEGER)

    # Objective function
    mdl.modelSense = GRB.MINIMIZE
    mdl.setObjective(quicksum(x[i, j] * distance_matrix[i, j] for (i, j) in Arcs))

    # Constraints
    add_constraints(mdl, num_items, num_couriers, Vertices, Items, Arcs, x, l, courier_assignment, aux_vars, item_loads, item_sizes)

    # Solver settings
    mdl.Params.TimeLimit = 300

    return mdl, x, courier_assignment, Items, Vertices



### Displaying the Optimization Results

In [368]:
def handle_solution(mdl, x, courier_assignment, Items, Vertices, num_couriers):
    tours = {}
    for i in Items:
        try:
            x[(0, i)].x > 0.1
        except AttributeError:
            print("Total distance: N/A")
            break
        if x[(0, i)].x > 0.9:
            for k in range(num_couriers):
                if int(str(courier_assignment[i][k])[-6:-4]) > 0:
                    temp = k
            temp += 1
            tours[temp] = []
            path = [0, i]
            while i != 0:
                j = i
                for k in Vertices:
                    if j != k and x[(j, k)].x > 0.9:
                        path.append(k)
                        i = k
            tours[temp].append(path)

    for key in tours.keys():
        path_str = str(tours[key])
        path_str = path_str.replace("[", "")
        path_str = path_str.replace("]", "")
        path_str = path_str.replace(", ", " > ")
        tours[key] = path_str

    tour_ids = [i for i in tours.keys()]
    print("Planned Tours:")

    for tour_id in sorted(tour_ids):
        print("Tour " + str(tour_id) + ":")
        print(tours[tour_id])

    print("---------------------------")
    try:
        print("Total distance:", int(mdl.ObjVal))
    except OverflowError:
        print("Total distance: N/A")





### Executing Item Assignment and Route Optimization for the MIP Model

In [369]:
def assign(num_items, num_couriers, item_loads, max_loads, distance_matrix):

    # Create distance matrix
    Dist = np.array(distance_matrix)

    # Create Gurobi model
    mdl, x, courier_assignment, Items, Vertices = create_model(num_items, num_couriers, item_loads, max_loads, Dist)

    # Optimize model
    mdl.optimize()

    # Print solution
    handle_solution(mdl, x, courier_assignment, Items, Vertices, num_couriers)


# assign(n, m, s, l, D)